In [ ]:
import os              
import tarfile
from six.moves import urllib
import pandas as pd   
import matplotlib.pyplot as plt
import numpy as np
import random as rd

In [ ]:
print("Current Working Directory:", os.getcwd())

HOUSING_PATH =  "../datasets/" 
HOUSING_NAME = "housing.csv"        
HOUSING_FULL = os.path.join (HOUSING_PATH, HOUSING_NAME)

housing = pd.read_csv (HOUSING_FULL, thousands = ',')                                           #housing = load_housing_data()


In [ ]:
# Fetch the data 

housing.head()

In [ ]:
housing.info ()

In [ ]:
housing.describe ()

In [ ]:
%matplotlib inline
housing.hist (bins = 50, figsize = (20, 15))
plt.show ()

In [ ]:
def split_train_test (data, test_ratio):
    shuffled_indices = np.random.permutation (len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices [:test_set_size]
    train_indices = shuffled_indices [test_set_size:]
    return data.iloc [train_indices], data.iloc [test_indices]    

In [ ]:
train_set, test_set = split_train_test (housing, 0.2)
print (len(train_set))
print(len(test_set))

In [ ]:
from zlib import crc32

def test_set_check(identifier, test_ratio):
 return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32
def split_train_test_by_id(data, test_ratio, id_column):
 ids = data[id_column]
